In [1]:
import os
import shutil
from tqdm import tqdm
from random import shuffle
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from glob import glob

import torch

from sklearn.model_selection import train_test_split

# Подготовка данных

In [6]:
TARGETS = {
    'Медведь': 0, 
    'Олень': 1, 
    'Рысь': 2, 
    'Лиса': 3,
    'Тигр': 4,
    'Леопард': 5,
    'Ласка': 6,
    'Волк': 7,
    'Орёл': 8,
    'Кабан': 9,
    'Сайгак': 10,
    'Белка': 11,
    'Енот': 12,
    'Человек': 13
}

# Человеков
# Волки, кабаны
# Барсуки
# Хз

## Камчатские данные + Other из Самары

In [64]:
box_data = pd.read_csv('boxed_all.csv')

In [65]:
len(glob('Распределённые животные/*/*'))

970

In [66]:
hand_data = pd.DataFrame()
hand_data['path'] = glob('Распределённые животные/*/*')
hand_data['file'] = [x.split('/')[-1] for x in glob('Распределённые животные/*/*')]
hand_data['type'] = [x.split('/')[-2] for x in glob('Распределённые животные/*/*')]

In [67]:
data = pd.merge(hand_data, box_data[['file', 'xmin', 'ymin', 'xmax', 'ymax']],
                on=['file'], how='left')

In [68]:
data = data[data.type != 'Плохо определяются'].reset_index(drop=True)

In [69]:
data.file.nunique() == data.shape[0]

True

In [70]:
def get_width(path):
    image = cv2.imread(path)
    return image.shape[1]

def get_height(path):
    image = cv2.imread(path)
    return image.shape[0]

widths = []
heights = []

for path in tqdm(data.path):
    widths.append(get_width(path))
    heights.append(get_height(path))
    
data['width'] = widths
data['height'] = heights

100%|█████████████████████████████████████████| 867/867 [00:56<00:00, 15.30it/s]


In [71]:
data.head()

,path,file,type,xmin,ymin,xmax,ymax,width,height
0,Распределённые животные/Медведь/209_IMG_0139_...,209_IMG_0139_S08.jpg,Медведь,857.0,367.0,1908.0,1027.0,1920,1080
1,Распределённые животные/Медведь/220_IMG_1865_...,220_IMG_1865_S07.jpg,Медведь,156.0,416.0,1572.0,1254.0,2048,1536
2,Распределённые животные/Медведь/50_D1456528_S...,50_D1456528_S07.jpg,Медведь,30.0,390.0,855.0,922.0,2048,1536
3,Распределённые животные/Медведь/24_IMAG0472_S...,24_IMAG0472_S08.jpg,Медведь,0.0,591.0,755.0,1225.0,2560,1920
4,Распределённые животные/Медведь/212_IMAG1942_...,212_IMAG1942_S08.jpg,Медведь,88.0,707.0,1029.0,1728.0,2560,1920


In [75]:
data['x_c'] = ( ((data['xmax'] + data['xmin']) / 2) / data['width'] ).apply(lambda x: max(0, min(1, x)))

data['y_c'] = ( ((data['ymax'] + data['ymin']) / 2) / data['height']).apply(lambda x: max(0, min(1, x)))

data['width_box'] = ((data['xmax'] - data['xmin']) / data['width']).apply(lambda x: max(0, min(1, x)))

data['height_box'] = ((data['ymax'] + data['ymin']) / data['height']).apply(lambda x: max(0, min(1, x)))

data['yolo'] = data.apply(lambda x: str(TARGETS[x[2]]) + ' ' 
                          + ' '.join([str(round(s, 4)) for s in x[-4:]]), axis=1)

In [78]:
data.head()

,path,file,type,xmin,ymin,xmax,ymax,width,height,x_c,y_c,width_box,height_box,yolo
0,Распределённые животные/Медведь/209_IMG_0139_...,209_IMG_0139_S08.jpg,Медведь,857.0,367.0,1908.0,1027.0,1920,1080,0.720052,0.645370,0.547396,1.000000,0 0.7201 0.6454 0.5474 1.0
1,Распределённые животные/Медведь/220_IMG_1865_...,220_IMG_1865_S07.jpg,Медведь,156.0,416.0,1572.0,1254.0,2048,1536,0.421875,0.543620,0.691406,1.000000,0 0.4219 0.5436 0.6914 1.0
2,Распределённые животные/Медведь/50_D1456528_S...,50_D1456528_S07.jpg,Медведь,30.0,390.0,855.0,922.0,2048,1536,0.216064,0.427083,0.402832,0.854167,0 0.2161 0.4271 0.4028 0.8542
3,Распределённые животные/Медведь/24_IMAG0472_S...,24_IMAG0472_S08.jpg,Медведь,0.0,591.0,755.0,1225.0,2560,1920,0.147461,0.472917,0.294922,0.945833,0 0.1475 0.4729 0.2949 0.9458
4,Распределённые животные/Медведь/212_IMAG1942_...,212_IMAG1942_S08.jpg,Медведь,88.0,707.0,1029.0,1728.0,2560,1920,0.218164,0.634115,0.367578,1.000000,0 0.2182 0.6341 0.3676 1.0


## Тигры и леопарды

In [79]:
tiger_train = pd.read_csv('data/Tiger/objects.csv')
leo_train = pd.read_csv('data/Leopard/objects.csv')

tiger_train['path'] = tiger_train.id.apply(lambda x: 'data/Tiger/'+x)
leo_train['path'] = leo_train.id.apply(lambda x: 'data/Leopard/'+x)

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

widths = []
heights = []

for path in tqdm(tiger_train.path):
    widths.append(get_width(path))
    heights.append(get_height(path))
    
tiger_train['width'] = widths
tiger_train['height'] = heights

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

widths = []
heights = []

for path in tqdm(leo_train.path):
    widths.append(get_width(path))
    heights.append(get_height(path))
    
leo_train['width'] = widths
leo_train['height'] = heights

100%|███████████████████████████████████████| 3517/3517 [03:43<00:00, 15.71it/s]


In [ ]:
# bbox - xmin, ymin, xmax, ymax
# yolo - x_c, y_c, width, height

# center_x = ( (xmax + xmin) / 2) / width
# center_y = ( (ymax + ymin) / 2) / height
# width = (xmax - xmin) / width
# height = (ymax - ymin) / height

In [80]:
tiger_train['x_c'] = tiger_train.bbox.apply(lambda x: ( (float(x.split(' ')[2]) + float(x.split(' ')[0])) / 2 ) )
tiger_train['x_c'] = (tiger_train['x_c'] / tiger_train['width']).apply(lambda x: max(0, min(1, x)))

tiger_train['y_c'] = tiger_train.bbox.apply(lambda x: ( (float(x.split(' ')[3]) + float(x.split(' ')[1])) / 2 ) )
tiger_train['y_c'] = (tiger_train['y_c'] / tiger_train['height']).apply(lambda x: max(0, min(1, x)))

tiger_train['width_box'] = tiger_train.bbox.apply( lambda x: (float(x.split(' ')[2]) - float(x.split(' ')[0])) )
tiger_train['width_box'] = (tiger_train['width_box'] / tiger_train['width']).apply(lambda x: max(0, min(1, x)))

tiger_train['height_box'] = tiger_train.bbox.apply(lambda x: (float(x.split(' ')[3]) - float(x.split(' ')[1])) )
tiger_train['height_box'] = (tiger_train['height_box'] / tiger_train['height']).apply(lambda x: max(0, min(1, x)))

tiger_train['yolo'] = tiger_train.apply(lambda x: '{} '.format(TARGETS['Тигр']) + 
                                        ' '.join([str(round(s, 4)) for s in x[-4:]]), axis=1)

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

leo_train['x_c'] = leo_train.bbox.apply(lambda x: ( (float(x.split(' ')[2]) + float(x.split(' ')[0])) / 2 ) )
leo_train['x_c'] = (leo_train['x_c'] / leo_train['width']).apply(lambda x: max(0, min(1, x)))

leo_train['y_c'] = leo_train.bbox.apply(lambda x: ( (float(x.split(' ')[3]) + float(x.split(' ')[1])) / 2 ) )
leo_train['y_c'] = (leo_train['y_c'] / leo_train['height']).apply(lambda x: max(0, min(1, x)))

leo_train['width_box'] = leo_train.bbox.apply( lambda x: (float(x.split(' ')[2]) - float(x.split(' ')[0])) )
leo_train['width_box'] = (leo_train['width_box'] / leo_train['width']).apply(lambda x: max(0, min(1, x)))

leo_train['height_box'] = leo_train.bbox.apply(lambda x: (float(x.split(' ')[3]) - float(x.split(' ')[1])) )
leo_train['height_box'] = (leo_train['height_box'] / leo_train['height']).apply(lambda x: max(0, min(1, x)))

leo_train['yolo'] = leo_train.apply(lambda x: '{} '.format(TARGETS['Леопард']) + 
                                    ' '.join([str(round(s, 4)) for s in x[-4:]]), axis=1)

## Формат йоло

In [88]:
data[['file', 'yolo']].rename(columns={'file': 'id'}).head()

,id,yolo
0,209_IMG_0139_S08.jpg,0 0.7201 0.6454 0.5474 1.0
1,220_IMG_1865_S07.jpg,0 0.4219 0.5436 0.6914 1.0
2,50_D1456528_S07.jpg,0 0.2161 0.4271 0.4028 0.8542
3,24_IMAG0472_S08.jpg,0 0.1475 0.4729 0.2949 0.9458
4,212_IMAG1942_S08.jpg,0 0.2182 0.6341 0.3676 1.0


In [85]:
leo_train[['id', 'yolo']].head()

,id,yolo
0,01020010_S02.jpg,5 0.5666 0.5382 0.3869 0.2594
1,212_IMG_0841_S02.jpg,5 0.543 0.7279 0.5449 0.5052
2,212_IMG_0840_S02.jpg,5 0.7336 0.6895 0.4937 0.4362
3,212_IMG_0839_S02.jpg,5 0.8469 0.6826 0.3032 0.4095
4,212_IMG_0838_S02.jpg,5 0.9446 0.6693 0.1079 0.3451


In [86]:
tiger_train[['id', 'yolo']].head()

,id,yolo
0,001_1.jpg,4 0.4392 0.4915 0.7339 0.4544
1,002_758.jpg,4 0.6543 0.5404 0.5957 0.3984
2,002_76.jpg,4 0.7946 0.6256 0.4078 0.5568
3,002_761.jpg,4 0.1584 0.5202 0.3159 0.2826
4,002_762.jpg,4 0.2786 0.5075 0.4224 0.2806


In [102]:
data_all = pd.concat([
    data.rename(columns={'file': 'id', 'type': 'class'})[['path', 'id', 'yolo', 'class']], 
    leo_train[['path', 'id', 'yolo', 'class']].sample(500), 
    tiger_train[['path', 'id', 'yolo', 'class']].sample(500)
], ignore_index=True)

In [104]:
train_all, val_all = train_test_split(data_all, test_size=0.15, stratify=data_all['class'])

In [105]:
for image_file in tqdm(train_all.id.unique()):
    
    temp_ann = train_all[train_all.id == image_file]
    image = cv2.imread(temp_ann.path.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'yolo_data/train/labels/'+image_file[:-4]+'.txt'
    image_file = 'yolo_data/train/images/'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|███████████████████████████████████████| 1584/1584 [02:15<00:00, 11.69it/s]


In [106]:
for image_file in tqdm(val_all.id.unique()):
    
    temp_ann = val_all[val_all.id == image_file]
    image = cv2.imread(temp_ann.path.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'yolo_data/val/labels/'+image_file[:-4]+'.txt'
    image_file = 'yolo_data/val/images/'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|█████████████████████████████████████████| 281/281 [00:25<00:00, 11.14it/s]


# Дополнение данных

In [110]:
TARGETS = {
    'Медведь': 0, 
    'Олень': 1, 
    'Рысь': 2, 
    'Лиса': 3,
    'Тигр': 4,
    'Леопард': 5,
    'Волк': 6,
    'Орёл': 7,
    'Кабан': 8,
    'Сайгак': 9,
    'Белка': 10,
    'Енот': 11,
    'Человек': 12,
    'Ласка': 13
}

## Датафрейм

In [85]:
files = glob('new_data/for_yolo/*/*')

In [86]:
new_data = pd.DataFrame()
new_data['path'] = files
new_data['id'] = [x.split('/')[-1] for x in files]
new_data['class'] = [x.split('/')[-2] for x in files]

In [87]:
new_data.shape

(1289, 3)

In [88]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /Users/neironeiro/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-4 torch 1.9.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [89]:
for i, path in tqdm(enumerate(new_data.path)):
    
    try:
        
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        pred = model(image).pandas().xyxy[0].sort_values('confidence', ascending=False)
            
        new_data.loc[i, 'xmin'] = int(pred.xmin.iloc[0])
        new_data.loc[i, 'ymin'] = int(pred.ymin.iloc[0])
        new_data.loc[i, 'xmax'] = int(pred.xmax.iloc[0])
        new_data.loc[i, 'ymax'] = int(pred.ymax.iloc[0])
        
        new_data.loc[i, 'animal'] = pred.name.iloc[0]
            
    except:
        pass

1289it [03:11,  6.74it/s]


In [90]:
new_data['box_square'] = (new_data.xmax - new_data.xmin) * (new_data.ymax - new_data.ymin)

In [91]:
new_data.animal.unique()

array(['bear', 'cat', 'cow', 'dog', nan, 'horse', 'sheep', 'giraffe', 'car', 'sports ball', 'bird', 'elephant', 'person', 'frisbee', 'zebra', 'surfboard', 'bench', 'fire hydrant'], dtype=object)

In [131]:
new_data_processed = new_data[(new_data.animal.isin(['bird', 'bear', 'dog', 'cat', 'sheep', 'horse', 'cow', 
                                                     'giraffe', 'bird', 'elephant', 'person', 'zebra']))
                              &(new_data.box_square > 0.08*1920*1080)].copy()

In [132]:
np.sort(new_data_processed['class'].unique())

array(['Белка', 'Волк', 'Енот', 'Кабан', 'Ласка', 'Лиса', 'Медведь', 'Олень', 'Орёл', 'Рысь', 'Сайгак', 'Человек'], dtype=object)

In [133]:
new_data_processed[new_data_processed['class'] == 'Ласка'].shape

(3, 9)

In [134]:
# new_data_processed = new_data_processed[new_data_processed['class'] != 'Ласка']

In [136]:
new_data_processed['class'].unique()[8]

'Сайгак'

In [135]:
new_data_processed['class'].unique()[10]

'Орёл'

In [137]:
saigak = new_data_processed['class'].unique()[8]
new_data_processed.loc[new_data_processed['class'] == saigak, 'class'] = 'Сайгак'

orel = new_data_processed['class'].unique()[10]
new_data_processed.loc[new_data_processed['class'] == orel, 'class'] = 'Орёл'

## В йоло

In [138]:
# def get_width(path):
#     image = cv2.imread(path)
#     return image.shape[1]

# def get_height(path):
#     image = cv2.imread(path)
#     return image.shape[0]

# widths = []
# heights = []

# for path in tqdm(new_data_processed.path):
    # widths.append(get_width(path))
    # heights.append(get_height(path))
    
new_data_processed['width'] = widths
new_data_processed['height'] = heights

In [141]:
new_data_processed['x_c'] = ( ((new_data_processed['xmax'] + new_data_processed['xmin']) / 2) / new_data_processed['width'] ).apply(lambda x: max(0, min(1, x)))

new_data_processed['y_c'] = ( ((new_data_processed['ymax'] + new_data_processed['ymin']) / 2) / new_data_processed['height']).apply(lambda x: max(0, min(1, x)))

new_data_processed['width_box'] = ((new_data_processed['xmax'] - new_data_processed['xmin']) / new_data_processed['width']).apply(lambda x: max(0, min(1, x)))

new_data_processed['height_box'] = ((new_data_processed['ymax'] + new_data_processed['ymin']) / new_data_processed['height']).apply(lambda x: max(0, min(1, x)))

new_data_processed['yolo'] = new_data_processed.apply(lambda x: str(TARGETS[x[2]]) + ' ' + ' '.join([str(round(s, 4)) for s in x[-4:]]), axis=1)

In [142]:
train_proc, val_proc = train_test_split(new_data_processed, test_size=0.15, stratify=new_data_processed['class'])

In [143]:
for image_file in tqdm(train_proc.id.unique()):
    
    temp_ann = train_proc[train_proc.id == image_file]
    image = cv2.imread(temp_ann.path.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'yolo_data/train/labels/'+image_file[:-4]+'.txt'
    image_file = 'yolo_data/train/images/'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|██████████| 976/976 [01:33<00:00, 10.42it/s]


In [144]:
for image_file in tqdm(val_proc.id.unique()):
    
    temp_ann = val_proc[val_proc.id == image_file]
    image = cv2.imread(temp_ann.path.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'yolo_data/val/labels/'+image_file[:-4]+'.txt'
    image_file = 'yolo_data/val/images/'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|██████████| 174/174 [00:16<00:00, 10.35it/s]


# Обучение YOLO

In [145]:
TARGETS = {
    'Медведь': 0, 
    'Олень': 1, 
    'Рысь': 2, 
    'Лиса': 3,
    'Тигр': 4,
    'Леопард': 5,
    'Волк': 6,
    'Орёл': 7,
    'Кабан': 8,
    'Сайгак': 9,
    'Белка': 10,
    'Енот': 11,
    'Человек': 12,
    'Ласка': 13
}

In [146]:
yaml_file = "yolov5/data/animal.yaml"
train_images_dir = os.path.join('..', 'yolo_data/train', 'images')
val_images_dir = os.path.join('..', 'yolo_data/val', 'images')

names_str = ''
for item in list(TARGETS.keys()):
    names_str = names_str + ", \'%s\'"%item
names_str = "names: ["+names_str[1:]+"]"

with open(yaml_file, "w") as wobj:
    wobj.write("train: %s\n"%train_images_dir)
    wobj.write("val: %s\n"%val_images_dir)
    wobj.write("nc: %d\n"%len(list(TARGETS.keys())))
    wobj.write(names_str+"\n")
    

In [147]:
!cd yolov5 && python train.py --img 512 --batch 8 --epochs 30 --data ../yolov5/data/animal.yaml --weights yolov5s.pt

# --weights ../yolov5/runs/train/exp4/weights/last.pt



train: weights=yolov5s.pt, cfg=, data=../yolov5/data/animal.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=8, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 2 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-122-gd885799 torch 1.9.0 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,

In [148]:
!cd yolov5 && python train.py --img 512 --batch 8 --epochs 30 --data ../yolov5/data/animal.yaml --weights ../yolov5/runs/train/exp5/weights/last.pt




train: weights=../yolov5/runs/train/exp5/weights/last.pt, cfg=, data=../yolov5/data/animal.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=8, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 2 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-122-gd885799 torch 1.9.0 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv